In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

events = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
training = pd.read_csv('labels_training_set.csv', low_memory = False)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
eventsWithLabel = events.merge(training, how='left', left_on='person', right_on='person')

In [4]:
eventsWithLabel['timestamp'] = pd.to_datetime(eventsWithLabel['timestamp'])

In [5]:
diccWeek = {0: "Monday",
            1: "Tuesday",
            2: "Wednesday",
            3: "Thursday",
            4: "Friday",
            5: "Saturday",
            6: "Sunday"}

In [6]:
#Obtengo los dias de la semana
eventsWithLabel['dayweek'] = eventsWithLabel.timestamp.dt.dayofweek
eventsWithLabel["dayweek"] = eventsWithLabel["dayweek"].map(lambda x: diccWeek.get(x))
#Obtengo la fecha
eventsWithLabel['fecha'] = eventsWithLabel.timestamp.dt.date
#Separo la marca
eventsWithLabel['marca'] = eventsWithLabel['model'].map(lambda x: str(x).split()[0])

In [7]:
groupbyCantEventos = eventsWithLabel.groupby('person', as_index = False)['event'].agg({'cantEventos': 'count'})
groupbyCantEventos.sort_values(by = 'cantEventos', ascending = False)
eventsWithLabel = pd.merge(eventsWithLabel, groupbyCantEventos, on = 'person')

In [8]:
#Filtro a la gente que no tiene el evento visited_site
personasConVisitas = eventsWithLabel.loc[eventsWithLabel['event'] == 'visited site']\
                             .drop_duplicates(subset = 'person', keep = 'first')['person'].tolist()
dataNotVisit = eventsWithLabel.loc[eventsWithLabel['person'].isin(personasConVisitas)]

In [9]:
eventsWithLabel2 = dataNotVisit.loc[dataNotVisit['event'] == 'visited site'].groupby(['person', 'fecha'], as_index = False).agg({'city': 'first', 'region': 'first', 'country': 'first', 'device_type': 'first', 'operating_system_version': 'first', 'channel': 'first', 'new_vs_returning': 'first'})

In [10]:
eventsWithLabel3 = pd.merge(dataNotVisit, eventsWithLabel2, on = ['person', 'fecha'])
eventsWithLabel3.drop(['region_x', 'device_type_x', 'operating_system_version_x', 'country_x', 'city_x', 'channel_x', 'new_vs_returning_x'], axis = 1, inplace = True)
eventsWithLabel3.rename(columns = {'region_y': 'region', 'device_type_y': 'device_type', 'operating_system_version_y': 'SO', 'city_y': 'city', 'country_y': 'country', 'channel_y': 'channel', 'new_vs_returning_y': 'new_vs_returning'}, inplace =True)

In [11]:
def obtenerSistema(so):
    sistema = so.split()
    if len(sistema) > 1:
        otro = sistema[1].split('.')[0]
        return sistema[0] + ' ' + otro
    else:
        return sistema[0]

In [12]:
eventsWithLabel3['SO'] = eventsWithLabel3['SO'].map(obtenerSistema)

In [13]:
eventsWithLabel3 = eventsWithLabel3.drop(columns=["url", "sku", "skus", "search_term", "staticpage", "campaign_source", "search_engine", "screen_resolution", "browser_version"])

In [14]:
#Empezamos con las sesiones.
eventsWithSessions = eventsWithLabel3.sort_values(by = ["person", "timestamp"])
eventsWithSessions["time_diff"] = (eventsWithSessions.timestamp - (eventsWithSessions.timestamp.shift())) / np.timedelta64(1, 's')
eventsWithSessions.loc[eventsWithSessions.person != eventsWithSessions.person.shift(), "time_diff"] = 0

In [15]:
eventsWithSessions["session_change"] = ((eventsWithSessions.event == "visited site") & (eventsWithSessions.time_diff > 1800.0)) | (eventsWithSessions.time_diff > 36000.0) | (eventsWithSessions.person != eventsWithSessions.person.shift())
eventsWithSessions["session_id"] = eventsWithSessions.groupby("person")["session_change"].cumsum()

In [16]:
#Obtener las personas con conversiones
dataConversion = eventsWithSessions.loc[eventsWithSessions['event'] == 'conversion']
personasCompraron = dataConversion.drop_duplicates(subset = 'person', keep = 'first')['person'].tolist()
dataPersonasCompraron = eventsWithSessions.loc[eventsWithSessions['person'].isin(personasCompraron)]

In [17]:
dataPersonasCompraron["aConversion"] = (dataPersonasCompraron.event == "conversion") | (dataPersonasCompraron.person != dataPersonasCompraron.person.shift())
dataPersonasCompraron["conversions"] = dataPersonasCompraron.groupby("person")["aConversion"].cumsum()
dataFirstConversion = dataPersonasCompraron.loc[dataPersonasCompraron.conversions == 1.0].groupby("person")["time_diff"].agg({"timeFirstConversion": "sum" })
eventsWithSessions = pd.merge(eventsWithSessions, dataFirstConversion, how='left', on = 'person')
eventsWithSessions['timeFirstConversion'] = eventsWithSessions['timeFirstConversion'].fillna(value='0.0')

/home/victorbelosevich/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/victorbelosevich/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/victorbelosevich/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package 

In [18]:
dataSession =  eventsWithSessions.groupby(["person", "session_id"], as_index=False)["time_diff"].agg({"sessionDuration": "sum" })
dataSession["sessionDuration"] = dataSession["sessionDuration"]/60

In [19]:
dataSessionGroupByPerson = dataSession.loc[dataSession.sessionDuration > 0.0].groupby("person")
dataSession = dataSessionGroupByPerson.agg({"sessionDuration": {"sessionDuration": "mean"}, "session_id": {"cantSessions": "count"}})
dataSession.columns = dataSession.columns.droplevel(0)

/home/victorbelosevich/.local/lib/python2.7/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [20]:
eventsWithSessions = pd.merge(eventsWithSessions, dataSession, how ='left', on = 'person')
eventsWithSessions["sessionDuration"] = eventsWithSessions["sessionDuration"].fillna(value = "0.0")
eventsWithSessions["cantSessions"] = eventsWithSessions["cantSessions"].fillna(value = "0.0")

In [21]:
#Funcion que realiza el oneHotEncoding
def oneHotEncoding(column, uniqueArray, dataFrame):
    label_encoder = LabelEncoder()
    label_encoder.fit(uniqueArray)
    integer_encoded = label_encoder.transform(dataFrame[column])
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    for i, item in enumerate(uniqueArray):
        dataFrame[item] = onehot_encoded[:, label_encoder.transform(uniqueArray)[i]]


In [22]:
eventsColor = eventsWithLabel3.loc[eventsWithLabel3["color"].notnull()][["color", "person"]]
eventsCondition = eventsWithLabel3.loc[eventsWithLabel3["condition"].notnull()][["condition", "person"]]
eventsStorage = eventsWithLabel3.loc[eventsWithLabel3["storage"].notnull()][["storage", "person"]]
eventsModel = eventsWithLabel3.loc[eventsWithLabel3["model"].notnull()][["model", "person"]]
eventsChannel = eventsWithLabel3.loc[eventsWithLabel3["channel"] != "Unknown"][["channel", "person"]]
eventsDevice = eventsWithLabel3.loc[eventsWithLabel3["device_type"] != "Unknown"][["device_type", "person"]]

In [23]:
eventsColor["color"] = eventsColor.color.str.split(" ", expand=True)
top20Colors = eventsColor['color'].value_counts().nlargest(20).index
top50models = eventsWithLabel3['model'].value_counts().nlargest(50).index

In [24]:
def colorGrouping(x):
    return x if x in top20Colors else 'OtroColor'

def modelGrouping(x):
    return x if x in top50models else 'OtroModelo'

In [25]:
eventsColor['color'] = eventsColor['color'].apply(colorGrouping)
eventsModel['model'] = eventsModel['model'].apply(modelGrouping)

In [26]:
eventsWithLabel3["SO"] = eventsWithLabel3.SO.str.split(" ", expand=True)

In [27]:
eventosUnicos = eventsWithLabel3.drop_duplicates(subset="event", keep="first").reset_index()["event"]
coloresUnicos = eventsColor.drop_duplicates(subset='color', keep='first').reset_index()["color"]
conditionUnicos = eventsCondition.drop_duplicates(subset='condition', keep='first').reset_index()["condition"]
storageUnicos = eventsStorage.drop_duplicates(subset='storage', keep='first').reset_index()["storage"]
modelosUnicos = eventsModel.drop_duplicates(subset='model', keep='first').reset_index()['model']
channelUnicos = eventsChannel.drop_duplicates(subset="channel", keep="first").reset_index()["channel"]
SOUnicos = eventsWithLabel3.drop_duplicates(subset="SO", keep="first").reset_index()["SO"]
deviceTypeUnicos = eventsDevice.drop_duplicates(subset="device_type", keep="first").reset_index()["device_type"]
daysWeekUnicos = eventsWithLabel3.drop_duplicates(subset="dayweek", keep="first").reset_index()["dayweek"]

In [28]:
oneHotEncoding("event", eventosUnicos, eventsWithLabel3)
oneHotEncoding("color", coloresUnicos, eventsColor)
oneHotEncoding("condition", conditionUnicos, eventsCondition)
oneHotEncoding("storage", storageUnicos, eventsStorage)
oneHotEncoding("model", modelosUnicos, eventsModel)
oneHotEncoding("channel", channelUnicos, eventsChannel)
oneHotEncoding("SO", SOUnicos, eventsWithLabel3)
oneHotEncoding("device_type", deviceTypeUnicos, eventsDevice)
oneHotEncoding("dayweek", daysWeekUnicos, eventsWithLabel3)

/home/victorbelosevich/.local/lib/python2.7/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [29]:
eventsFinale = eventsWithLabel3.drop(columns=["timestamp", "dayweek","event", "cantEventos","condition", "model", "storage", "color", "fecha", "SO", "marca", "city", "country", "region", "channel", "device_type", "new_vs_returning" ])

In [30]:
eventsColor1 = eventsColor.groupby("person").sum()
eventsCondition1 = eventsCondition.groupby("person").sum()
eventsDevice1 = eventsDevice.groupby("person").sum()
eventsModel1 = eventsModel.groupby("person").sum()
eventsStorage1 = eventsStorage.groupby("person").sum()
eventsChannel1 = eventsChannel.groupby("person").sum()

In [31]:
eventsWithSessionsFilter = eventsWithSessions.groupby("person")[["timeFirstConversion", "sessionDuration", "cantSessions"]].agg("first")

In [32]:
testDF = eventsFinale.loc[eventsFinale['label'].isnull()]
trainingDF = eventsFinale.loc[eventsFinale['label'].notnull()]

In [33]:
testDFFinal = testDF.drop(columns=["label"]).groupby("person").sum()
trainingDFFinal = trainingDF.groupby("person").sum()

In [34]:
testDFFinal = pd.merge(testDFFinal, eventsWithSessionsFilter, how="left" , on = "person")
testDFFinal = pd.merge(testDFFinal, eventsColor1, how="left" , on = "person")
testDFFinal = pd.merge(testDFFinal, eventsChannel1, how="left" , on = "person")
testDFFinal = pd.merge(testDFFinal, eventsCondition1, how="left" , on = "person")
testDFFinal = pd.merge(testDFFinal, eventsDevice1, how="left" , on = "person")
testDFFinal = pd.merge(testDFFinal, eventsModel1, how="left" , on = "person")
testDFFinal = pd.merge(testDFFinal, eventsStorage1, how="left" , on = "person")

In [41]:
testDFFinal = testDFFinal.fillna(value=0.0)

In [42]:
testDFFinal.head()

,viewed product,search engine hit,checkout,searched products,generic listing,visited site,ad campaign hit,brand listing,lead,staticpage,...,Motorola Moto G2 3G Dual,Samsung Galaxy Gran Prime Duos TV,32GB,64GB,16GB,128GB,256GB,8GB,4GB,512MB
person,,,,,,,,,,,,,,,,,,,,,
00091926,372.0,0.0,2.0,0.0,0.0,34.0,15.0,25.0,0.0,0.0,...,0.0,0.0,132.0,80.0,104.0,48.0,10.0,0.0,0.0,0.0
00091a7a,3.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
000ba417,153.0,1.0,6.0,0.0,14.0,6.0,1.0,24.0,0.0,0.0,...,0.0,6.0,20.0,1.0,115.0,0.0,1.0,22.0,1.0,0.0
000e4d9e,339.0,5.0,1.0,0.0,17.0,13.0,19.0,17.0,0.0,0.0,...,0.0,0.0,209.0,21.0,108.0,1.0,1.0,0.0,0.0,0.0
000e619d,28.0,3.0,1.0,6.0,8.0,5.0,6.0,11.0,0.0,0.0,...,4.0,0.0,11.0,3.0,8.0,0.0,1.0,6.0,0.0,0.0


In [37]:
trainingDFFinal = pd.merge(trainingDFFinal, eventsWithSessionsFilter, how="left" , on = "person")
trainingDFFinal = pd.merge(trainingDFFinal,  eventsColor1, how="left" , on = "person")
trainingDFFinal = pd.merge(trainingDFFinal, eventsChannel1, how="left" , on = "person")
trainingDFFinal = pd.merge(trainingDFFinal, eventsCondition1, how="left" , on = "person")
trainingDFFinal = pd.merge(trainingDFFinal, eventsDevice1, how="left" , on = "person")
trainingDFFinal = pd.merge(trainingDFFinal, eventsModel1, how="left" , on = "person")
trainingDFFinal = pd.merge(trainingDFFinal, eventsStorage1, how="left" , on = "person")

In [43]:
trainingDFFinal = trainingDFFinal.fillna(value=0.0)

In [44]:
trainingDFFinal.head()

,label,viewed product,search engine hit,checkout,searched products,generic listing,visited site,ad campaign hit,brand listing,lead,...,Motorola Moto G2 3G Dual,Samsung Galaxy Gran Prime Duos TV,32GB,64GB,16GB,128GB,256GB,8GB,4GB,512MB
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
000c79fe,0.0,3.0,1.0,1.0,9.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
001802e4,0.0,4.0,0.0,1.0,4.0,4.0,1.0,5.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0
0019e639,0.0,189.0,13.0,15.0,11.0,28.0,19.0,29.0,165.0,0.0,...,63.0,2.0,14.0,0.0,98.0,0.0,0.0,94.0,0.0,0.0
001ca5ee,0.0,52.0,6.0,1.0,0.0,8.0,15.0,7.0,7.0,0.0,...,0.0,0.0,11.0,10.0,20.0,8.0,4.0,0.0,0.0,0.0


In [45]:
#Creamos los csv ya listo para entrenar a los algoritmos y para testear que tan buenos son.
trainingDFFinal.to_csv('setEntrenamiento.csv')
testDFFinal.to_csv('setTesteo.csv')